In [100]:
import os
from urllib.parse import parse_qs
from IPython.display import Markdown, display
import ipywidgets as widgets
import csv
import json
import html

In [83]:
ROOT_PATH = "../"

In [84]:
query_string = os.getenv("QUERY_STRING")
row_id = None
if query_string:
    if "method" in parse_qs(query_string):
        method_index = int(parse_qs(query_string)["method"][0])
    if "results" in parse_qs(query_string):
        result_filepath = os.path.join(ROOT_PATH, parse_qs(query_string)["results"][0])
    if "assertions" in parse_qs(query_string):
        assertion_filepath = os.path.join(
            ROOT_PATH, parse_qs(query_string)["assertions"][0]
        )

In [85]:
# row_id = 0
# result_filepath = ".//tests_package/ressources/fixes_llm_test.csv"
# assertion_filepath = ".//tests_package/ressources/llm_test.csv"
# method_index = 0
# result_filepath = ".//results_llm/stats_llm_libraries_sample10_context.csv"
# assertion_filepath = ".//samples/non_verified_libraries_sample_10.csv"

In [86]:
result = []
with open(result_filepath, "r") as file:
    csv_reader = csv.DictReader(file)
    for index, row in enumerate(csv_reader):
        if row["Index"] == str(method_index):
            result.append(row)

with open(assertion_filepath, "r") as file:
    csv_reader = csv.DictReader(file)
    for index, row in enumerate(csv_reader):
        if index == method_index:
            assertion = row
            break

In [87]:
def extract_dafny_functions(dafny_code, name):
    inside_function = False
    current_function = ""
    brace_count = 0

    lines = dafny_code.split("\n")

    for line in lines:
        if f"lemma {name}" in line or f"method {name}" in line:
            inside_function = True
            current_function += line + "\n"
            brace_count += line.count("{") - line.count("}")
        elif inside_function:
            current_function += line + "\n"
            brace_line_count = line.count("{") - line.count("}")
            brace_count += brace_line_count

            # hack to identify when a function finishes
            # {} if it is an empty function
            # otherwise expect } and with a brace count to 0 (since we are finishing the func)
            # and not an even number of braces on the line (the odd is the one to finish the func)
            if (
                brace_count == 0
                and "}" in line
                and (brace_line_count != 0 or "{}" in line)
            ):
                inside_function = False
                return current_function


def remove_warning(message):
    lines = message.split("\n")

    # Find the index of the first line containing an error
    error_index = next(
        (i for i, line in enumerate(lines) if "Error" in line), len(lines)
    )
    filtered_lines = lines[error_index:]
    return "\n".join(filtered_lines)

In [88]:
def display_in_markdown_codeblock(content):
    return Markdown(f"```\n{content}\n```")


def display_content_in_markdown_codeblock(file_path):
    with open(file_path, "r") as file:
        content = file.read()
    return display_in_markdown_codeblock(content)

In [99]:
display(
    Markdown(
        f'# Assertion : `{assertion["Assertion"][:-1]}` in {assertion["Original Method"]}'
    )
)
display(Markdown(f"## Original Method"))
base_file = assertion["Original File"]
if not os.path.isabs(base_file):
    base_file = os.path.join(ROOT_PATH, base_file)
with open(base_file, "r") as file:
    base_code = file.read()
og_method = extract_dafny_functions(base_code, assertion["Original Method"])

display(display_in_markdown_codeblock(og_method))
display(Markdown(f"## Error Message"))
error_file = os.path.join(ROOT_PATH, result[0]["Original Error Message File"])
with open(error_file, "r") as file:
    error_message = file.read()
display(display_in_markdown_codeblock(remove_warning(error_message)))

# Assertion : `assert unit1 == bop(unit1, unit2)` in UnitIsUnique

## Original Method

```
lemma UnitIsUnique<T(!new)>(bop: (T, T) -> T, unit1: T, unit2: T)
  requires IsUnital(bop, unit1)
    requires IsUnital(bop, unit2)
      ensures unit1 == unit2
{
    assert unit1 == bop(unit1, unit2);
    assert unit2 == bop(unit2, unit1);
}

```

## Error Message

```
tests_package/ressources/unital_assert.dfy(21,0): Error: a postcondition could not be proved on this return path
   |
21 | {
   | ^

tests_package/ressources/unital_assert.dfy(20,14): Related location: this is the postcondition that could not be proved
   |
20 |       ensures unit1 == unit2
   |               ^^^^^^^^^^^^^^


Dafny program verifier finished with 0 verified, 1 error

```

In [102]:
# accordion = widgets.Accordion(
#     children=[widgets.Output() for _ in range(len(result))],
#     titles=[f"Try {i}" for i in range(len(result))],
# )
accordion = widgets.Accordion(
    children=[widgets.Output() for _ in range(len(result))],
    titles=[f"Try {i} feedback" if row["Feedback"]=="True" else f"Try {i}" for i, row in enumerate(result)],
    # titles=[f"Try {i}" for i in range(len(result))],
)

try_sections = []
for row in result:
    # Create separate accordions for each section
    diff_accordion = widgets.Accordion(children=[widgets.Output()], titles=("Diff",))
    result_accordion = widgets.Accordion(children=[widgets.Output()], titles=["Result"])
    error_accordion = widgets.Accordion(children=[widgets.Output()], titles=["Error"])
    prompt_accordion = widgets.Accordion(children=[widgets.Output()], titles=["Prompt"])
    method_accordion = widgets.Accordion(
        children=[widgets.Output()], titles=["New Method"]
    )

    # Set up display for each accordion section
    with diff_accordion.children[0]:
        display(Markdown(f"## Assertion generated"))
        display(display_in_markdown_codeblock(row["Diff"]))

    with result_accordion.children[0]:
        display(Markdown(f'## Result: {row["New Method Result"]}'))

    if (
        row["New Method Result"] == "Error"
        and row["New Method Error Message File"] != ""
    ):
        error_file = os.path.join(ROOT_PATH, row["New Method Error Message File"])
        with open(error_file, "r") as file:
            error_message = file.read()
        display(Markdown(f"## Error Message"))
        with error_accordion.children[0]:
            display(display_in_markdown_codeblock(remove_warning(error_message)))
    elif row["Error message"] != "":
        with error_accordion.children[0]:
            display(Markdown(f"## Error Message"))
            if not os.path.isabs(row["Error message"]):
                error_file = os.path.join(ROOT_PATH, row["Error message"])
            with open(error_file, "r") as file:
                error_message = file.read()
            display(display_in_markdown_codeblock(error_message))

    with prompt_accordion.children[0]:
        display(Markdown(f"## Prompt"))
        prompt_file = row["Prompt File"]
        if not os.path.isabs(prompt_file):
            prompt_file = os.path.join(ROOT_PATH, prompt_file)
        with open(prompt_file, "r") as file:
            prompt = json.load(file)
        for item in prompt:
            # content = item["content"].replace("\n", "<br>")
            # content = html.escape(item["content"]).replace("\n", "<br>")
            content = (
                html.escape(item["content"])
                .replace(" ", "&nbsp;")
                .replace("\n", "<br>")
            )

            if item["role"] == "system":
                display(Markdown(f"### {item['role']}\n> {content}"))
            else:
                display(Markdown(f"### {item['role']}\n \n{content}\n"))
        # for item in prompt:
        #     if item['role'] == 'system':
        #         display(Markdown(f"### {item['role']}\n> {str(item['content'])}"))
        #     elif item['role'] == 'user':
        #         display(Markdown(f"### {item['role']}\n {str(item['content'])} "))
        #     elif item['role'] == 'assistant':
        #         display(Markdown(f"### {item['role']}\n {item['content']} "V))
        # display(display_content_in_markdown_codeblock(prompt_file))

    with method_accordion.children[0]:
        display(Markdown(f"## New Method"))
        base_file = row["New Method File"]
        if base_file != "default_file_path":
            if not os.path.isabs(base_file):
                base_file = os.path.join(ROOT_PATH, base_file)
            with open(base_file, "r") as file:
                base_code = file.read()
            og_method = extract_dafny_functions(base_code, assertion["Original Method"])
            display(display_in_markdown_codeblock(og_method))

    # display(diff_accordion)
    # display(result_accordion)
    # display(error_accordion)
    # display(prompt_accordion)
    # display(method_accordion)
    try_sections.append(
        widgets.VBox(
            [
                diff_accordion,
                result_accordion,
                error_accordion,
                prompt_accordion,
                method_accordion,
            ]
        )
    )

accordion.children = try_sections
display(accordion)

Accordion(children=(VBox(children=(Accordion(children=(Output(),), titles=('Diff',)), Accordion(children=(Outp…